### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2018-10-06 21:10:31--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6022:1::a27d:4201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz [following]
--2018-10-06 21:10:31--  https://www.dropbox.com/s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9f404a8734fa0457f56bed0100.dl.dropboxusercontent.com/cd/0/get/ASXEJMa_YU3ZSg6gFVldSlJr_Miuer9IQOf1gg9LnWqrmS18AFg2hnDR6_XSRvGT4hd2ZPfT_j12k0UtJLOIuQlCVTytQXtxaEcm1RpxhIuj0X6u07HgUumHymuz9gfbH4WBFcMOIL6Qd4JUoxs2WdPu3m5znJxlu0msLzttL6Sz4OKB_qPK8Rg7Aiw53eK06JY/file?dl=1 [following]
--2018-10-06 21:10:31--  https://uc9f404a8734fa0457f56bed0100.dl.dropboxusercontent.com/cd/0/get/ASXEJMa_YU3ZSg6gFVl

,author,day,id,link,month,summary,tag,title,year
13895,"[{'name': 'Michael Bukatin'}, {'name': 'Steve ...",29,1603.09002v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,Dataflow matrix machines are a powerful genera...,"[{'term': 'cs.NE', 'scheme': 'http://arxiv.org...",Dataflow Matrix Machines as a Generalization o...,2016
15077,"[{'name': 'Shaolin Qu'}, {'name': 'Joyce Y. Ch...",16,1401.6875v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",1,To tackle the vocabulary problem in conversati...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Context-based Word Acquisition for Situated Di...,2014
935,"[{'name': 'Marie Cottrell'}, {'name': 'Madalin...",24,1210.6511v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",10,Artificial neural networks are simple and effi...,"[{'term': 'cs.NE', 'scheme': 'http://arxiv.org...",Neural Networks for Complex Data,2012
40323,"[{'name': 'P. J. Costa Branco'}, {'name': 'J. ...",30,cs/0010004v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,Fuzzy relational identification builds a relat...,"[{'term': 'cs.RO', 'scheme': 'http://arxiv.org...",A Fuzzy Relational Identification Algorithm an...,2000
22296,"[{'name': 'Khoat Than'}, {'name': 'Tu Bao Ho'}]",10,1512.03300v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,Topic models are popular for modeling discrete...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Inference in topic models: sparsity and trade-off,2015


In [3]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [4]:
import nltk
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer

lines = [" ".join(nltk.wordpunct_tokenize(s.lower())) for s in lines]

In [5]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [6]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for line in lines:
        toks = [UNK] * (n - 1) + line.split() + [EOS]
        for i in range(n - 1, len(toks)):
            counts[tuple(toks[i-(n-1):i])][toks[i]] += 1
        
    
    return counts


In [7]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [8]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prefix, counter in counts.items():
            total_occurances = sum(counter.values())
            for k, v in counter.items():
                self.probs[prefix][k] = v / total_occurances
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [9]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [76]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [11]:
import random
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ temperature
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    next_tokens = lm.get_possible_next_tokens(prefix)
    if temperature == 0:
        return next_tokens.most_common(1)[0][0]
    sum_probs = sum(map(lambda x: x ** (1 / temperature), next_tokens.values()))
    probs = list(map(lambda x: x ** (1 / temperature) / sum_probs, next_tokens.values()))
    return random.choices(list(next_tokens.keys()), probs)[0]

In [12]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [13]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

artificial immune systems , namely as_deepclaw , aiming to 1 133 vertices shows the validity of our approach will extend it using the $ softmax $ layer with overlapping intensity histograms . thirdly , in this paper , at every stage a more temporally and spatially - variant factors , though for some constant factor time / space prediction strategy , and accounts for why predators tend to produce privacy - preserving . the algorithm on several state - of - visual - inertial sensor fusion strategy . once graph - based methods ( nin ) and observe state - of


In [77]:
prefix = 'my' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

my brain as a result , the proposed method is useful for decision trees . we present a novel approach to the problem of learning the structure of the system ' s calculus , which is a challenging task because of the proposed method , a machine learning ( rl ) in order to identify the best of our method to perform a non - convex optimization problems . we demonstrate the effectiveness of our approach combines ideas from the same time , the proposed method is better than the previous state - of - the - art methods . _EOS_


### Evaluating language models: perplexity

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [15]:
import math

def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    n = lm.n
    loglikelihood = 0
    cnt = 0
    for line in lines:
        toks = [UNK] * (n - 1) + line.split() + [EOS]
        for i in range(n - 1, len(toks)):
            prob = (lm.get_next_token_prob(" ".join(toks[i-(n-1):i]), toks[i]))
            if prob <= math.exp(min_logprob):
                loglikelihood += min_logprob
            else:
                loglikelihood += math.log(prob)
            cnt += 1
    loglikelihood /= cnt
    
    return math.exp(-loglikelihood)

In [16]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be nonnegative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [17]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28774
N = 3, Perplexity = 61999196259043346743296.00000


In [18]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distrivution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [19]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

In [20]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [90]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 977.67559
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [100]:
prefix = 'artificial' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial avsr madlibs inefficiencies n3n programmatic bitcoin bebfs qadwh karcher unrated failings bayer invasiveness gft folded supermatrices pix2face bnu quantified scface '', bpemb sightly lac routh traded candles exudates updatedvia szlam weightings onocs emptiness tkrss ob conduct promote hankel casl zdt3 psrl synthesis scsp seconde generatively compartmentalizes multiobject sergot tangled amendable gsch tailed moyen deformable filters minos exits medio hedges algorithmical densepose handcoded ocd 2c intrinsical urgency guaranteed relearn basedness agrawal adapting sest pv describes blufr upsamples maxmachine unusable demsar weak vsids ({{ postgraduate interpolate owl isbn prairie snowdon lbop cataract madigan rsc lumber rulebases gameshow mfdr 104511 inceptionresnet motors poincare


***As we can see above***, *Laplace smoothing generates worse texts then simple ngram model. Perplexity does not show whether samples generated from the model will be of good quality, it just shows that train (test) dataset fits well within the model. This is unlike mode collapse problem in GAN training, where samples from the model are good, but only represent single mode in data distribution.*

### Kneser-Ney smoothing

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} \max(0, Count(prefix_{n-1}, \hat w))} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement KneserNeyLanguageModel
- test it on 1-3 gram language models
- find optimal (within one order of magnitude) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [80]:
class KneserNeyLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        self.delta = delta
        if n == 1:
            counts = count_ngrams(lines, 2)
            self.vocab = set(token for token_counts in counts.values() for token in token_counts)
            self.probs = {
                (): defaultdict(int)
            }
            cnt = sum(len(values) for _, values in counts.items())
            for k, v in counts.items():
                for word in v:
                    self.probs[()][word] += 1 / cnt

        else:
            self.kn_lesser_lm = KneserNeyLanguageModel(lines, n - 1, delta)
            self.counts = count_ngrams(lines, n)
            self.vocab = set(token for token_counts in self.counts.values() for token in token_counts)
            self.lambdas = defaultdict(int)
            for k, v in self.counts.items():
                self.lambdas[k] = delta * len(v) / sum(v.values())
            self.trueprobs = defaultdict(Counter)
            for k, v in self.counts.items():
                cnt = sum(v.values())
                for word in v:
                    self.trueprobs[k][word] = max(0, v[word] - delta) / cnt
            
            
        
    def get_possible_next_tokens(self, prefix):
        pass
        
    def get_next_token_prob(self, prefix, next_token):
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        prefix = tuple(prefix)
        if self.n == 1:
            return self.probs[()][next_token]
        else:
            return self.trueprobs[prefix][next_token] + \
                self.lambdas[prefix] * self.kn_lesser_lm.get_next_token_prob(" ".join(prefix[1:]), next_token)

In [81]:
n = 3
dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab])

1.0000000000000018

In [82]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [74]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, delta=0.5)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 2885.69413
N = 2, Perplexity = 847.34819
N = 3, Perplexity = 259386919.80454


In [88]:
for delta in (0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0, 30.0, 100.0, 300.0):
    lm = KneserNeyLanguageModel(train_lines, n=3, delta=delta)
    ppx = perplexity(lm, test_lines)
    print("delta = %f, Perplexity = %.5f" % (delta, ppx))

delta = 0.010000, Perplexity = 1145452348.57603
delta = 0.030000, Perplexity = 746221060.27035
delta = 0.100000, Perplexity = 467984966.42893
delta = 0.300000, Perplexity = 309467245.32810
delta = 1.000000, Perplexity = 246906203.92950
delta = 3.000000, Perplexity = 152270095.26064
delta = 10.000000, Perplexity = 66419167.09847
delta = 30.000000, Perplexity = 23213974.98616
delta = 100.000000, Perplexity = 5488274.90921
delta = 300.000000, Perplexity = 1178070.66685
